### Setup

In [17]:
import os
import time
import boto3
import itertools
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import sagemaker
from sagemaker import get_execution_role

from sklearn.tree import DecisionTreeClassifier
import joblib
import tarfile

In [26]:
sess = boto3.Session()
sm = sess.client("sagemaker")
role = get_execution_role()
sagemaker_session = sagemaker.Session(boto_session=sess)
bucket = sagemaker_session.default_bucket() # 'data-aman-use-case'
prefix = "traffic-logisitcs-experiment"

### Dataset

In [3]:
X = pd.read_csv("X.csv")
y = pd.read_csv("y.csv")

In [4]:
data_dir = os.path.join(os.getcwd(), "data")
os.makedirs(data_dir, exist_ok=True)

train_dir = os.path.join(os.getcwd(), "data/train")
os.makedirs(train_dir, exist_ok=True)

test_dir = os.path.join(os.getcwd(), "data/test")
os.makedirs(test_dir, exist_ok=True)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=1
)

In [6]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [7]:
np.save(os.path.join(train_dir, "x_train.npy"), X_train)
np.save(os.path.join(test_dir, "x_test.npy"), X_test)
np.save(os.path.join(train_dir, "y_train.npy"), y_train)
np.save(os.path.join(test_dir, "y_test.npy"), y_test)

In [8]:
s3_inputs_train = sagemaker.Session().upload_data(
    path="data/train", bucket=bucket, key_prefix=prefix + "/train"
)
s3_inputs_test = sagemaker.Session().upload_data(
    path="data/test", bucket=bucket, key_prefix=prefix + "/test"
)
inputs = {"train": s3_inputs_train, "test": s3_inputs_test}
print(inputs)

{'train': 's3://sagemaker-us-east-2-190183093544/traffic-logisitcs-experiment/train', 'test': 's3://sagemaker-us-east-2-190183093544/traffic-logisitcs-experiment/test'}


### Step 1 - Set up the Experiment

### Create an Experiment

In [9]:
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker

In [10]:
logistics_experiment = Experiment.create(
    experiment_name=f"traffic-logisitcs-{int(time.time())}",
    description="Training on traffic logisitcs dataset",
    sagemaker_boto_client=sm,
)
print(logistics_experiment )

Experiment(sagemaker_boto_client=<botocore.client.SageMaker object at 0x7f93ef092890>,experiment_name='traffic-logisitcs-1659633471',description='Training on traffic logisitcs dataset',tags=None,experiment_arn='arn:aws:sagemaker:us-east-2:190183093544:experiment/traffic-logisitcs-1659633471',response_metadata={'RequestId': '8b203462-6922-4092-a14e-519c71ff68d8', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '8b203462-6922-4092-a14e-519c71ff68d8', 'content-type': 'application/x-amz-json-1.1', 'content-length': '100', 'date': 'Thu, 04 Aug 2022 17:17:50 GMT'}, 'RetryAttempts': 0})


### Step 2 - Track Experiment

In [11]:
hyperparam_options = {"max_depth": [6, 8, 10]}

hypnames, hypvalues = zip(*hyperparam_options.items())
trial_hyperparameter_set = [dict(zip(hypnames, h)) for h in itertools.product(*hypvalues)]
trial_hyperparameter_set

[{'max_depth': 6}, {'max_depth': 8}, {'max_depth': 10}]

In [12]:
from sagemaker.sklearn import SKLearn, SKLearnModel

In [13]:
run_number = 1
for trial_hyp in trial_hyperparameter_set:
    # Combine static hyperparameters and trial specific hyperparameters
    hyperparams = trial_hyp

    # Create unique job name with hyperparameter and time
    time_append = int(time.time())
    hyp_append = "-".join([str(elm).replace(".", "-") for elm in trial_hyp.values()])
    training_job_name = f"traffic-logisitcs-training-{hyp_append}-{time_append}"
    trial_name = f"trial-traffic-logisitcs-training-{hyp_append}-{time_append}"
    trial_desc = f"traffic-logisitcs-run-{run_number}"

    # Create a new Trial and associate Tracker to it
    logistics_trial = Trial.create(
        trial_name=trial_name,
        experiment_name=logistics_experiment.experiment_name,
        sagemaker_boto_client=sm,
        tags=[{"Key": "trial-desc", "Value": trial_desc}],
    )

    # Create an experiment config that associates training job to the Trial
    experiment_config = {
        "ExperimentName": logistics_experiment.experiment_name,
        "TrialName": logistics_trial.trial_name,
        "TrialComponentDisplayName": training_job_name,
    }

    metric_definitions = [
        {"Name": "f1", "Regex": "f1: ([0-9\\.]+)"},
        {"Name": "accuracy", "Regex": "accuracy: ([0-9\\.]+)"},
        {"Name": "precision", "Regex": "precision: ([0-9\\.]+)"},
        {"Name": "recall", "Regex": "recall: ([0-9\\.]+)"},
    ]

    # Create a TensorFlow Estimator with the Trial specific hyperparameters
    logistics_estimator = SKLearn(
        entry_point="traffic-logisitcs.py",
        source_dir="code",
        role=sagemaker.get_execution_role(),
        instance_count=1,
        instance_type="ml.m5.large",
        framework_version="0.23-1",
        hyperparameters=hyperparams,
        py_version="py3",
        metric_definitions=metric_definitions,
        enable_sagemaker_metrics=True,
        tags=[{"Key": "trial-desc", "Value": trial_desc}],
    )

    # Launch a training job
    logistics_estimator.fit(
        inputs, job_name=training_job_name, experiment_config=experiment_config
    )

    # give it a while before dispatching the next training job
    time.sleep(2)
    run_number = run_number + 1

INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: traffic-logisitcs-training-6-1659633477


2022-08-04 17:17:58 Starting - Starting the training job...
2022-08-04 17:18:22 Starting - Preparing the instances for trainingProfilerReport-1659633478: InProgress
......
2022-08-04 17:19:27 Downloading - Downloading input data......
2022-08-04 17:20:28 Training - Downloading the training image......
2022-08-04 17:21:23 Training - Training image download completed. Training in progress.2022-08-04 17:21:19,771 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2022-08-04 17:21:19,776 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-08-04 17:21:19,790 sagemaker_sklearn_container.training INFO     Invoking user training script.
2022-08-04 17:21:20,209 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-08-04 17:21:20,238 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-08-04 17:21:20,270 sagemaker-training-toolkit INFO     No GPUs detected (

INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: traffic-logisitcs-training-8-1659633733


2022-08-04 17:22:15 Starting - Starting the training job...
2022-08-04 17:22:32 Starting - Preparing the instances for trainingProfilerReport-1659633735: InProgress
......
2022-08-04 17:23:42 Downloading - Downloading input data.........
2022-08-04 17:25:09 Training - Downloading the training image..2022-08-04 17:25:26,326 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2022-08-04 17:25:26,330 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-08-04 17:25:26,344 sagemaker_sklearn_container.training INFO     Invoking user training script.
2022-08-04 17:25:26,747 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-08-04 17:25:26,762 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-08-04 17:25:26,782 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-08-04 17:25:26,799 sagemaker-training-toolkit INFO     

INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: traffic-logisitcs-training-10-1659633960


2022-08-04 17:26:01 Starting - Starting the training job...
2022-08-04 17:26:25 Starting - Preparing the instances for trainingProfilerReport-1659633961: InProgress
......
2022-08-04 17:27:28 Downloading - Downloading input data......
2022-08-04 17:28:29 Training - Downloading the training image.....2022-08-04 17:29:07,391 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2022-08-04 17:29:07,395 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-08-04 17:29:07,420 sagemaker_sklearn_container.training INFO     Invoking user training script.
2022-08-04 17:29:07,889 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-08-04 17:29:07,907 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-08-04 17:29:07,927 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-08-04 17:29:07,944 sagemaker-training-toolkit INFO     

### Compare the model training runs for experiment

In [14]:
from sagemaker.analytics import ExperimentAnalytics

experiment_name = logistics_experiment.experiment_name

trial_component_analytics = ExperimentAnalytics(
    sagemaker_session=sagemaker_session, experiment_name=experiment_name
)
trial_comp_ds_jobs = trial_component_analytics.dataframe()
trial_comp_ds_jobs

,TrialComponentName,DisplayName,SourceArn,SageMaker.ImageUri,SageMaker.InstanceCount,SageMaker.InstanceType,SageMaker.VolumeSizeInGB,max_depth,sagemaker_container_log_level,sagemaker_job_name,...,test - MediaType,test - Value,train - MediaType,train - Value,SageMaker.DebugHookOutput - MediaType,SageMaker.DebugHookOutput - Value,SageMaker.ModelArtifact - MediaType,SageMaker.ModelArtifact - Value,Trials,Experiments
0,traffic-logisitcs-training-10-1659633960-aws-t...,traffic-logisitcs-training-10-1659633960,arn:aws:sagemaker:us-east-2:190183093544:train...,257758044811.dkr.ecr.us-east-2.amazonaws.com/s...,1.0,ml.m5.large,30.0,10.0,20.0,"""traffic-logisitcs-training-10-1659633960""",...,None,s3://sagemaker-us-east-2-190183093544/traffic-...,None,s3://sagemaker-us-east-2-190183093544/traffic-...,None,s3://sagemaker-us-east-2-190183093544/,None,s3://sagemaker-us-east-2-190183093544/traffic-...,[trial-traffic-logisitcs-training-10-1659633960],[traffic-logisitcs-1659633471]
1,traffic-logisitcs-training-8-1659633733-aws-tr...,traffic-logisitcs-training-8-1659633733,arn:aws:sagemaker:us-east-2:190183093544:train...,257758044811.dkr.ecr.us-east-2.amazonaws.com/s...,1.0,ml.m5.large,30.0,8.0,20.0,"""traffic-logisitcs-training-8-1659633733""",...,None,s3://sagemaker-us-east-2-190183093544/traffic-...,None,s3://sagemaker-us-east-2-190183093544/traffic-...,None,s3://sagemaker-us-east-2-190183093544/,None,s3://sagemaker-us-east-2-190183093544/traffic-...,[trial-traffic-logisitcs-training-8-1659633733],[traffic-logisitcs-1659633471]
2,traffic-logisitcs-training-6-1659633477-aws-tr...,traffic-logisitcs-training-6-1659633477,arn:aws:sagemaker:us-east-2:190183093544:train...,257758044811.dkr.ecr.us-east-2.amazonaws.com/s...,1.0,ml.m5.large,30.0,6.0,20.0,"""traffic-logisitcs-training-6-1659633477""",...,None,s3://sagemaker-us-east-2-190183093544/traffic-...,None,s3://sagemaker-us-east-2-190183093544/traffic-...,None,s3://sagemaker-us-east-2-190183093544/,None,s3://sagemaker-us-east-2-190183093544/traffic-...,[trial-traffic-logisitcs-training-6-1659633477],[traffic-logisitcs-1659633471]


### Reproduce the best model

In [15]:
d_tree = DecisionTreeClassifier(max_depth=10)
d_tree.fit(X_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=10, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [18]:
path = os.path.join("model.joblib")
joblib.dump(d_tree, path)

['model.joblib']

In [19]:
with tarfile.open('model.tar.gz', "w:gz") as tar:
    tar.add('model.joblib')

In [27]:
session = boto3.Session()
s3 = session.resource('s3')

local_path = 'model.tar.gz' 
s3_filename =  'model.tar.gz' 
result = s3.Bucket(bucket).upload_file(local_path, s3_filename)

In [28]:
model_artifact = 's3://' + bucket + '/' + s3_filename
model_artifact

's3://sagemaker-us-east-2-190183093544/model.tar.gz'